In [1]:
import re
import numpy as np
import pandas as pd
from decimal import *
from sklearn.metrics import recall_score, precision_score, f1_score

In [2]:
with open('pred.out', 'r', encoding='utf-8') as p:
    pred = [p.replace('\n', '') for p in p.readlines()]
with open('feats.test', 'r', encoding='utf-8') as t:
    ref = [t.replace('\n', '') for t in t.readlines()]

In [3]:
pred_labels = []
for i in range(len(pred)):
    pred_labels.append(re.findall(r'^([0-9]+?) ', pred[i])[0])
    
ref_labels = []
for i in range(len(ref)):
    ref_labels.append(re.findall(r'^([0-9]+?) ', ref[i])[0])

In [4]:
df = pd.DataFrame(columns=['pred_labels', 'ref_labels'])
df['pred_labels'] = pred_labels
df['ref_labels'] = ref_labels
df[:3]

,pred_labels,ref_labels
0,7,11
1,8,8
2,9,9


In [5]:
_acc = np.sum(df['pred_labels'] == df['ref_labels']) / len(df)
_acc

0.6656

In [6]:
class_to_num_pred = dict()
class_to_num_ref = dict()
for cl in range(1, 15):
    class_to_num_pred[cl] = list(df['pred_labels'][(df['pred_labels'].astype(int) == cl)].index)
    class_to_num_ref[cl] = list(df['ref_labels'][(df['ref_labels'].astype(int) == cl)].index)

In [7]:
def evaluate(class_num, total):
    _pred = class_to_num_pred[class_num]
    _ref = class_to_num_ref[class_num]
    TP = 0
    for ind in _pred:
        if ind in _ref:
            TP += 1
    FN = len(_ref) - TP
    FP = len(_pred) - TP
    TN = (total - len(_pred)) - FN
    assert (total - len(_pred)) - FN == (total - len(_ref)) - FP
    
#     print('TP: {}, FP: {}\n'.format(TP, FP) +
#           'FN: {}, TN: {}'.format(FN, TN))
    
    acc = (TP + TN) / float(total)
    P = TP / float(TP + FP)
    R = TP / float(TP + FN)
    F1 = (2 * P * R) / float(P + R) 
    
    return acc, P, R, F1

In [8]:
evaluate(1, len(df))

(0.9632, 0.71875, 0.9019607843137255, 0.8)

In [9]:
m_F1 = 0
for i in range(1, 15):
    _, _, _, F1 = evaluate(i, len(df))
    m_F1 += F1
m_F1 = m_F1 / float(14)

In [10]:
with open('Eval.txt', 'w+', encoding='utf-8') as e:
    e.write('Accuracy = ' + str(round(_acc, 3)) + '\n')
    e.write('Macro-F1 = ' + str(round(m_F1, 3)) + '\n')
    e.write('Results per class:\n')
    for i in range(1, 15):
        _, P, R, F1 = evaluate(i, len(df))
        e.write('{}: P={} R={} F={}'.format(i, round(P, 3), round(R, 3), round(F1, 3)) + '\n')